In [1]:
import numpy as np
import nibabel as nib
from glob import glob
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
from models.detr import DETR
from models.segmentation import DETRsegm
from hubconf import detr_resnet101_panoptic, detr_resnet101
from torchvision.transforms import Resize

In [2]:
from models.position_encoding import PositionEmbeddingSine
from util.misc import NestedTensor

In [3]:
from models.hub.resnet import _resnet

In [4]:
# class DETRsegm_3D(nn.Module):
#     def __init__(self):
#         detr = detr_resnet101_panoptic()
#         conv

In [5]:
def sort_func(path):
    path_id = int(path.split('/')[-1].split('_')[1])
    return path_id

In [6]:
def show_image_and_label(image, label):
    fig, axs = plt.subplots(nrows=1,ncols=2, squeeze=False,figsize=(12, 12))
    axs[0, 0].imshow(image)
    axs[0, 0].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[], title='Image')
    axs[0, 1].imshow(label)
    axs[0, 1].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[], title='Label')

In [7]:
class DatasetForSegmentation(Dataset):
    
    def __init__(self, image_paths, label_paths):
        self.image_paths = image_paths
        self.label_paths = label_paths
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, i):
        image_np_array = nib.load(image_paths[i]).get_fdata()
        image_torch_tensor = torch.from_numpy(image_np_array)
        label_np_array = nib.load(image_paths[i]).get_fdata()
        label_torch_tensor = torch.from_numpy(label_np_array)
        return image_torch_tensor, label_np_array

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
# path = '/home/francisco/workspace/ImageCHD_dataset'

In [10]:
# image_paths = glob(f'{path}/*image.nii.gz',recursive=True)
# label_paths = glob(f'{path}/*label.nii.gz',recursive=True)

In [11]:
# image_paths.sort(key=sort_func)
# label_paths.sort(key=sort_func)

In [12]:
# dset = DatasetForSegmentation(image_paths,label_paths)

In [13]:
# inpt, outp = dset[2]

In [14]:
# model = _resnet();
# model = model.half()
# model.to(device);

In [15]:
# inpt_right_shape = inpt.transpose(2,0).unsqueeze(0).unsqueeze(0).float()
# new_size = [int(inpt_right_shape.shape[2]/3),int(inpt_right_shape.shape[3]/3),int(inpt_right_shape.shape[4]/3)]
# inpt_resized = inpt_right_shape.resize_((1,1,new_size[0],new_size[1],new_size[2]))

In [16]:
# cv_3d = torch.nn.Conv3d(1,3,(1,3,3),padding = 1)
# result = cv_3d(inpt_resized)
# result_cuda = result.to(device)
# result_2 = model(result_cuda.half())

In [17]:
# detr = detr_resnet101_panoptic();
# detr.eval();

***

In [18]:
detr = detr_resnet101();#_panoptic()
detr.eval();
detr.to(device);
resnet_3d = _resnet();
resnet_3d.to(device);

/home/francisco/anaconda3/envs/PyTorch/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/francisco/anaconda3/envs/PyTorch/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
im = torch.ones((1, 1, 32, 64, 64),device=device)
# im = torch.ones((1, 3, 128, 128),device=device)

In [20]:
detr.backbone[0] = resnet_3d

In [21]:
# list_of_tensors = NestedTensor(im,torch.ones((2,32,128,128),dtype=torch.int,device=device))

In [22]:
# pos = PositionEmbeddingSine(num_pos_feats=128)

In [23]:
# poss = pos(list_of_tensors)

In [24]:
preds = detr(im)

/home/francisco/workspace/CHD_Classifier_by_Francisco_Lourenço/models/position_encoding.py:84: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


torch.Size([1, 384, 8, 16, 16]) torch.Size([1, 8, 16, 16]) torch.Size([100, 384]) torch.Size([1, 384, 8, 16, 16])



torch.Size([1, 1, 32, 128, 128]) torch.Size([1, 32, 128, 128])

torch.Size([1, 2048, 8, 32, 32]) torch.Size([1, 32, 128, 128])

torch.Size([1, 2048, 4, 4]) torch.Size([1, 4, 4])


torch.Size([1, 256, 4, 4])



torch.Size([1, 2048, 8, 32, 32]) torch.Size([1, 32, 128, 128])


torch.Size([1, 384, 32, 128, 128])

torch.Size([1, 256, 8, 32, 32]) torch.Size([1, 32, 128, 128]) torch.Size([100, 256]) torch.Size([1, 256, 32, 128, 128])

2D:

torch.Size([1, 256, 4, 4]) torch.Size([1, 4, 4]) torch.Size([100, 256]) torch.Size([1, 256, 4, 4])

In [25]:
preds

{'pred_logits': tensor([[[ 0.1987, -0.7765, -0.6291,  ..., -0.3947,  0.3182, -0.3763],
          [ 0.1879, -0.7659, -0.6195,  ..., -0.4126,  0.3078, -0.3858],
          [ 0.1922, -0.7814, -0.6339,  ..., -0.3981,  0.3134, -0.3794],
          ...,
          [ 0.1962, -0.7872, -0.6320,  ..., -0.3997,  0.3174, -0.3821],
          [ 0.1923, -0.7853, -0.6414,  ..., -0.4099,  0.3012, -0.3793],
          [ 0.1767, -0.7735, -0.6227,  ..., -0.3992,  0.3092, -0.3966]]],
        device='cuda:0', grad_fn=<SelectBackward0>),
 'pred_boxes': tensor([[[0.5040, 0.4942, 0.5530, 0.5042],
          [0.5043, 0.4936, 0.5538, 0.5045],
          [0.5037, 0.4935, 0.5536, 0.5043],
          [0.5036, 0.4937, 0.5543, 0.5046],
          [0.5039, 0.4942, 0.5541, 0.5045],
          [0.5041, 0.4943, 0.5544, 0.5044],
          [0.5035, 0.4941, 0.5541, 0.5045],
          [0.5040, 0.4947, 0.5545, 0.5047],
          [0.5044, 0.4942, 0.5546, 0.5046],
          [0.5045, 0.4935, 0.5543, 0.5045],
          [0.5046, 0.4936, 0.

***

***

In [ ]:
""" The forward expects a NestedTensor, which consists of:
       - samples.tensor: batched images, of shape [batch_size x 3 x H x W]
       - samples.mask: a binary mask of shape [batch_size x H x W], containing 1 on padded pixels

In [117]:
class PositionEmbeddingSine(nn.Module):
    """
    This is a more standard version of the position embedding, very similar to the one
    used by the Attention is all you need paper, generalized to work on images.
    """
    def __init__(self, num_pos_feats=64, temperature=10000, normalize=False, scale=None):
        super().__init__()
        self.num_pos_feats = num_pos_feats
        self.temperature = temperature
        self.normalize = normalize
        if scale is not None and normalize is False:
            raise ValueError("normalize should be True if scale is passed")
        if scale is None:
            scale = 2 * np.pi
        self.scale = scale

    def forward(self, tensor_list: NestedTensor):
        x = tensor_list.tensors
        mask = tensor_list.mask
        assert mask is not None
        not_mask = ~mask
        z_embed = not_mask.cumsum(1, dtype=torch.float32)
        y_embed = not_mask.cumsum(2, dtype=torch.float32)
        x_embed = not_mask.cumsum(3, dtype=torch.float32)
        
        if self.normalize:
            eps = 1e-6
            z_embed = z_embed / (z_embed[:, -1:, :, :] + eps) * self.scale
            y_embed = y_embed / (y_embed[:, :, -1:, :] + eps) * self.scale
            x_embed = x_embed / (x_embed[:, :, :, -1:] + eps) * self.scale

        dim_t = torch.arange(self.num_pos_feats, dtype=torch.float32, device=x.device)
        dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)
        
        pos_x = x_embed[:, :, :, :, None] / dim_t
        pos_y = y_embed[:, :, :, :, None] / dim_t
        pos_z = z_embed[:, :, :, :, None] / dim_t
        pos_x = torch.stack((pos_x[:, :, :, :, 0::2].sin(), pos_x[:, :, :, :, 1::2].cos()), dim=5).flatten(4)
        pos_y = torch.stack((pos_y[:, :, :, :, 0::2].sin(), pos_y[:, :, :, :, 1::2].cos()), dim=5).flatten(4)
        pos = torch.cat((pos_y, pos_x), dim=4).permute(0, 4, 1, 2, 3)
        return pos

In [118]:
pos = PositionEmbeddingSine(num_pos_feats=128)
list_of_tensors = NestedTensor(im,torch.ones((1,32,128,128),dtype=torch.int,device=device))
poss = pos(list_of_tensors)

/tmp/ipykernel_55046/3997029984.py:33: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


In [119]:
poss.shape

torch.Size([1, 256, 32, 128, 128])